# Social Distancing Detection - AWS Marketplace Model Package

This app will count the number of people in the frame, number of peoople with safe and unsafe distance using deep Learning.

This notebook is provided as an additional resource to assist you in deploying facemask detection

## Subscribe to the Model Package
Before you can deploy the model, your account needs to be subscribed to it.

1. Open the Model Package listing page
2. Read the product overview section and Highlights section to understand the value proposition of our model package.
3. View usage information and then additional resources sections. 
4. Click on Continue to Subscribe button & then Click on "Accept Offer" button on the next page.
5. Next, Continue to Configuration button becomes activated and when you click on the button, you will see that a Product Arn will appear.
6. You have successfully subscribed to our model package. You can find it in your [SageMaker console](https://console.aws.amazon.com/sagemaker/home?region=us-east-1#/model-packages/my-subscriptions).
7. Close the page and continue executing this notebook.


## Imports

In [9]:
import sagemaker as sage
import os
import boto3
from sagemaker import get_execution_role
from sagemaker import transformer
from sagemaker import model

role = get_execution_role()

## Create the session

The session remembers our connection parameters to Amazon SageMaker. We'll use it to perform all of our SageMaker operations.

In [10]:
sess = sage.Session()

account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
region

'us-east-2'

## Create Model

Now we use the License Plate Detection & Recognition Model Package to create a model.

In [11]:
from src.model_package_arns import ModelPackageArnProvider

In [12]:
from sagemaker import ModelPackage
modelpackage_arn = ModelPackageArnProvider.get_model_package_arn(sess.boto_region_name)

# Define predictor wrapper class
def predict_wrapper(endpoint, session):
    return sage.RealTimePredictor(endpoint, session, content_type='video/mp4')

# Create a deployable model
model = ModelPackage(role=role,
                                      model_package_arn = modelpackage_arn,
                                      sagemaker_session = sess,
                                      predictor_cls = predict_wrapper)
# ARN
print(modelpackage_arn)

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


arn:aws:sagemaker:us-east-2:165536952993:model-package/social-1595969497


## Live Inference Endpoint

Now we demonstrate the creation of an endpoint for live inference

In [13]:
# Deploy the model
predictor = model.deploy(1, 'ml.m4.xlarge', endpoint_name='social-detection-endpoint')

--------------!

## Testing on a Single video File 

The model takes a video file as input. We give the location of the sample input file and prepare it for payload. An example has been shown in the following code shell.


In [14]:
file_name = 'data/lobby.mp4'

In [15]:
endpoint_name='social-inference'

### Preparing the Input Payload and Viewing the Response

In [18]:
# Open the file and read the image into a bytearray.
with open(file_name, "rb") as image:
  b = bytearray(image.read())

# Perform a prediction
result = predictor.predict(b).decode('utf-8')

with open("data/output.txt", "w") as text_file:
    text_file.write(result)
    
# View the prediction
print(result)

                           TOTAL COUNT  ... SAFE COUNT
2020-07-28 21:38:50.367724           3  ...          0
2020-07-28 21:38:50.372058           3  ...          0
2020-07-28 21:38:50.375093           3  ...          0
2020-07-28 21:38:51.204435           3  ...          0
2020-07-28 21:38:51.207972           3  ...          0
2020-07-28 21:38:51.211006           3  ...          0
2020-07-28 21:38:51.966729           3  ...          0
2020-07-28 21:38:51.970245           3  ...          0
2020-07-28 21:38:51.973269           3  ...          0
2020-07-28 21:38:52.769110           3  ...          0
2020-07-28 21:38:52.772623           3  ...          0
2020-07-28 21:38:52.775606           3  ...          0
2020-07-28 21:38:53.534984           4  ...          0
2020-07-28 21:38:53.538554           4  ...          0
2020-07-28 21:38:53.541516           4  ...          0
2020-07-28 21:38:53.544440           4  ...          0
2020-07-28 21:38:54.300142           4  ...          0
2020-07-28

### Delete the Endpoint


Now that you have successfully performed a real-time inference, you do not need the endpoint any more. you can terminate the same to avoid being charged.


In [ ]:
predictor = sage.RealTimePredictor(endpoint_name, sess, content_type='video/mp4')
predictor.delete_endpoint(delete_endpoint_config=True)